## Q1.

Add methods `__iter__` to your project Time Series class to iterate over values, a method `itertimes` to iterate over times, a method `itervalues` to iterate over values, and a method `iteritems` to iterate over time-value pairs. (This is a similar interface to python dictionaries). To test these, check both the types of the results and the answers you expect.

In [1]:
import numpy as np
from lazy import *
class TimeSeries():
    '''
    """
Help on package TimeSeries:

NAME
    TimeSeries

DESCRIPTION
    TimeSeries
    =====
    
    Provides
      1. An sequence or any iterable objects
    
    How to use the documentation
    ----------------------------
    Documentation is available in two forms: docstrings provided
    with the code, and a loose standing reference guide, available from
    `the TimeSeries homepage <https://github.com/cs207-project>`_.
    
    We recommend exploring the docstrings using
    `IPython <http://ipython.scipy.org>`_, an advanced Python shell with
    TAB-completion and introspection capabilities.  See below for further
    instructions.
    
    The docstring examples assume that `numpy` has been imported as `np`::  
      
    
    
     |  Methods inherited from builtins.RuntimeWarning:
     |  
     |  __init__(self, *args, **kwargs)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |      Stors a TimeSeries in self.TimeSeries_
     |    
     |  __repr__(self, /)
     |      Return a printable sequence shown in python list format containing all values in [self].
     |  
     |  __str__(self, /)
     |      Return a printable abbreviated sequence of maximum first 100 entrees.
     |  
     |  __getitem__(self, index)
     |      Return self[index]
     |
     |  __setitem__(self, index, values)
     |      Set self[index] = values
     |
     |  __len__(self)
     |      Return len(self.TimeSeries_)
     Examples
     --------
     >>> a = TimeSeries(np.arange(0,100))
     >>> len(a)
     100
     >>> a[2]
     2
     >>> a[2]=3
     >>> a[2]
     3
    '''
    def __init__(self, times, values):
        if (iter(times) and iter(values)):
            # reorder according to Time step
            idx = np.argsort(times)
            times = np.array(times)[idx]
            values = np.array(values)[idx]

            self._TimeSeries=np.vstack((times,values))
            self._vindex = 0
            self._values = self._TimeSeries[1]
            self._times = self._TimeSeries[0]
    
    @property
    @lazy
    def lazy(self):
        return self
    def itervalues(self):
        for v in self._values:
            yield v
    def itertimes(self):
        for t in self._times:
            yield t
    def iteritemes(self):
        for t,v in zip(self._times,self._values):
            yield (t,v)
    def __len__(self):
        return len(self._TimeSeries[0])
    
    def __contains__(self, time):
        index = np.where(self._TimeSeries[0]==time)
        return index[0].size>0
            
    
    def __getitem__(self,time):
        if (time in self):
            index = np.where(self._TimeSeries[0]==time)
            return self._TimeSeries[1][index]
        else:
            print ("no time point at t={0}".format(time))

    def __setitem__(self,time,value):
        if (time in self):
            index = np.where(self._TimeSeries[0]==time)
            self._TimeSeries[1][index]=value
        else:
            print ("no time point at t={0}".format(time))
            
    def __iter__(self):
        return iter(self._TimeSeries[1])
    
    def __repr__(self):
        return "%r"%(self._TimeSeries)
    
    def __str__(self):
        className = type(self).__name__
        if len(self._TimeSeries)>100:
            return "%s" %('['+(str(self._TimeSeries[:99]))[1:-1]+'...'+']')
        else:
            return "%s" %(self._TimeSeries)
        
    def __eq__(self, other):
        return np.array_equal(self._TimeSeries, other._TimeSeries)
        
    def values(self):
        return self._values
    
    def times(self):
        return self._times
    
    def mean(self):        
        if(len(self._values) == 0):
            raise ValueError("cant calculate mean of length 0 list")
        return np.mean(self._values)
    
    def median(self):
        if(len(self._values) == 0):
            raise ValueError("cant calculate median of length 0 list")
        return np.median(self._values)
    
    def interpolate(self, times):
        new_values = []
        for time in times:
            if time > self._times[-1]: # over the rightest boundary
                new_values.append(self._values[-1])
            elif time < self._times[0]: # over the leftest boundary
                new_values.append(self._values[0])
            elif time in self._times:
                new_values.append(self.__getitem__(time))
            else : #within boundary
                for i in range(len(self._times)):
                    if self._times[i] > time:
                        left_value = self._values[i-1]
                        right_value = self._values[i]
                        left_time = self._times[i-1]
                        right_time = self._times[i]
                        #interpolate
                        new_values.append(left_value + (right_value - left_value)/(right_time - left_time)*(time - left_time))
                        break
        return TimeSeries(times, new_values)


In [12]:
a = TimeSeries([],[])
a.itervalues()
import collections
list(a.itervalues())
list(a.iteritemes())
isinstance(list(a.iteritemes()),collections.Iterable)

True

## Q2.

An online mean and standard deviation algorithm.

Below is a function to generate a potentially infinite stream of 1-D data.

In [3]:
from random import normalvariate, random
from itertools import count
def make_data(m, stop=None):
    for _ in count():
        if stop and _ > stop:
            break
        yield 1.0e09 + normalvariate(0, m*random() )
        

Here is an implementation of an online mean algorithm..see http://www.johndcook.com/blog/standard_deviation/ and the link to http://www.johndcook.com/blog/2008/09/26/comparing-three-methods-of-computing-standard-deviation/ in-between. (Convince yourselves of the formulas...)

In [4]:
def online_mean(iterator):
    n = 0
    mu = 0
    for value in iterator:
        n += 1
        delta = value - mu
        mu = mu + delta/n
        yield mu

We use out generator functions to implement iterators:

In [5]:
g = make_data(5, 10)
(g)

<generator object make_data at 0x107534938>

In [6]:
g = online_mean(make_data(5, 100))
#print(type(g))
list(g)

[999999998.569577,
 999999998.9547269,
 999999998.5693382,
 999999998.9681697,
 999999999.0351012,
 999999998.9440804,
 999999999.1586812,
 999999999.547137,
 999999999.394719,
 999999999.4517989,
 999999999.0886197,
 999999999.3127072,
 999999999.4174309,
 999999999.7032504,
 999999999.8133506,
 999999999.9172896,
 1000000000.1188653,
 1000000000.0920857,
 1000000000.1420126,
 1000000000.1932214,
 1000000000.2142547,
 1000000000.0615169,
 999999999.8423303,
 999999999.7980988,
 1000000000.0519217,
 1000000000.2908087,
 1000000000.2800788,
 1000000000.274911,
 1000000000.1384115,
 1000000000.1494014,
 1000000000.1448034,
 1000000000.0558128,
 999999999.9906037,
 1000000000.1155831,
 1000000000.0226128,
 1000000000.0231346,
 1000000000.15749,
 1000000000.173452,
 1000000000.1709545,
 1000000000.1197692,
 1000000000.1161027,
 1000000000.1166652,
 1000000000.2381734,
 1000000000.3513114,
 1000000000.2462964,
 1000000000.1712711,
 1000000000.1141527,
 1000000000.1342933,
 1000000000.212947

### 2.1

Implement the standard deviation algorithm as a generator function as

```python
def online_mean_dev(iterator):
    BLA BLA
    if n > 1:
        stddev = math.sqrt(dev_accum/(n-1))
        yield (n, value, mu, stddev)
```

In [9]:
# your code here
import math
def online_mean_dev(iterator):
    n = 0
    mu = 0
    dev_accum = 0
    for value in iterator:
        n+=1
        delta = value - mu
        delta_dev1 = value - mu
        mu = mu + delta/n
        delta_dev2 = value - mu
        if n > 1:
            dev_accum += delta_dev1*delta_dev2
            stddev = math.sqrt(dev_accum/(n-1))
            yield (n, value, mu, stddev)
        #yield (n,value,mu,0)

Here we make 100000 element data, and run this iterator on it (imagine running this on a time-series being slowly read from disk

In [14]:
data_with_stats = online_mean_dev(make_data(5, 100))


## Q3.

Let's do Anomaly detection. Write a routine `is_ok`:

```python
def is_ok(level, t)
```

which takes a tuple like the one yielded by your code above and returns True if the value is inbetween `level`-$\sigma$ of the mean.

In [9]:
def is_ok(level,t):
    dist = level-t[3]
    if ((t[1]<t[2]+dist) and (t[1]>t[2]-dist)):
        return True
    return False
    

We use this function to create a predicate passed through to `itertools.filterfalse` which is then used to obtain an iterator on the anomalies.

In [10]:
from itertools import filterfalse
pred = lambda t: is_ok(5, t)
anomalies = filterfalse(pred, data_with_stats)

We materialize the anomalies...

In [11]:
list(anomalies)#materialize

[(2, 999999999.6905417, 999999996.1809835, 4.963264763413398),
 (3, 999999999.7704003, 999999997.3774558, 4.075737479432299),
 (6, 1000000000.3518113, 999999997.9800653, 3.0237570341120765),
 (8, 1000000002.1813898, 999999998.6387855, 2.9528901010830517),
 (9, 999999993.8537444, 999999998.1071142, 3.189621234625334),
 (11, 1000000004.1299673, 999999998.6720132, 3.379208551823878),
 (14, 999999989.1900831, 999999998.1070923, 3.9323468997906876),
 (15, 999999994.4971915, 999999997.8664322, 3.902253945918048),
 (16, 1000000002.6290523, 999999998.164096, 3.9534886916206267),
 (17, 1000000001.63816, 999999998.3684527, 3.9195843608998953),
 (22, 999999996.0858971, 999999998.3427336, 3.486155259483629),
 (23, 999999994.7125949, 999999998.1849015, 3.489098627728195),
 (24, 1000000001.6217804, 999999998.3281047, 3.4837746120621333),
 (25, 1000000002.6597905, 999999998.5013722, 3.518739884590559),
 (27, 999999994.3767586, 999999998.3756804, 3.476718581806754),
 (31, 999999994.4960095, 999999998.

## To think of, but not hand in

What kinds of anomalies will this algorithm pick up? What kinds would a shorter "window" of anomaly detection, like 100 points around the time in question pick? How might you create an algorithm which does window based averaging? (hint: the window size is small compared to the time series size). 

Finally think a bit of how you might implement all of this in a production environment..remember that data streaming in might get backed up when you handle an anomaly.

(Some inspiration might accrue if you look at the docs for `collections.deque`).